Импортируем необходимые библиотеки

In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time

Определяем обычную ссылку на страницу и формат ссылки для пагинации

In [24]:
base_url = "https://lifehacker.ru/"
base_url_tech_page = "https://lifehacker.ru/topics/technology/?page={}"

Так как теги класса формируеютс при помощи js, необходимо установить Selenium

In [ ]:
%pip install -q google-colab-selenium

In [26]:
import google_colab_selenium as gs
from selenium.webdriver.chrome.service import Service

Собраем ссылки на статьи по 10 страницам

In [27]:
parsed_urls = []
pages_to_parse = 10


for page_num in range(1, pages_to_parse + 1):
  url = base_url_tech_page.format(page_num)
  driver = gs.Chrome()
  driver.get(url)
  time.sleep(5)
  #response = requests.get(driver.page_source)
  soup = BeautifulSoup(driver.page_source, 'lxml')
  raw_items = soup.find_all('a', class_='lh-small-article-card__link') # получаем ссылку на статью из карточки
  parsed_urls += [item.get('href') for item in raw_items]
  driver.quit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Собираем заголовки и содержание статей

In [20]:
result = []

for url in tqdm(parsed_urls):
  article = {}

  article_url = f'{base_url}{url}' # подставляем часть ссылки, ведущую к материалу
  response = requests.get(article_url)
  soup = BeautifulSoup(response.text, 'lxml')

  # Безопасное извлечение заголовка
  title_tag = soup.find('h1', class_='article-card__title')
  article['title'] = title_tag.text.strip() if title_tag else None

  # Безопасное извлечение текста
  content_tag = soup.find('article', class_='single-article__post-content')
  article['text'] = content_tag.text.strip() if content_tag else None

  result.append(article)

100%|██████████| 300/300 [08:12<00:00,  1.64s/it]


Делаем оформление

In [21]:
pd.set_option('display.max_colwidth', 400)

Вывод результатов

In [22]:
data = pd.DataFrame(result)
data.head()

,title,text
0,"В самолёт и поезд: 20 бесплатных мобильных игр, которым не нужен интернет","Долгая дорога — отличный повод отключиться от суеты и немного поиграть. Но не всегда под рукой есть стабильный интернет. На этот случай мы подготовили подборку бесплатных мобильных игр, которые работают офлайн. Всё, что нужно — зарядить телефон или планшет и заранее скачать то, что понравилось. Выбирайте!Defense Zone 3 HDХардкорный Tower Defense.Рейтинг в Google Play: 4,7.\nРейтинг в App Store..."
1,Как выбрать бюджетный игровой ноутбук,"Какой ноутбук можно считать игровымОчевидно, что игровым можно назвать то устройство, на котором будет комфортно запускать современные игры, требовательные к производительности. Но что подразумевается под понятием «комфортно»? Разберём ряд важных для этого параметров.FPSFPS (Frame per Second — «количество кадров в секунду») один из основных показателей, влияющих на комфорт игры. Чем больше кад..."
2,Лучшие смартфоны апреля,"Realme GT7 — холодный флагман с новым Dimensity 9400+Изображение: Realme Дисплей: AMOLED, 6,8 дюйма, 2 800 × 1 280 пикселей.\nПроцессор: MediaTek Dimensity 9400+ (3 нм).\nКамера: 50 Мп (широкоугольная) + 8 Мп (ультраширокоугольная); фронтальная — 16 Мп.\nПамять: 12/256, 12/512, 16/512, 16/1024 ГБ.\nАккумулятор: 7 200 мА·ч; проводная зарядка 100 Вт.\nСистема: Realme UI 6.0 (Android 15).\nВыносл..."
3,Xiaomi выпустила умную электронную скакалку с дисплеем,"Компания Xiaomi анонсировала Mijia Smart Skipping Rope 2 — улучшенную версию своей умной скакалки. От обычной она отличается отсутствием традиционной верёвки — её заменяют небольшие утяжелители, имитирующие сопротивления при прыжках. Его можно адаптировать под рост пользователя.Изображение: Xiaomi От предыдущего поколения скакалки Mijia Smart Skipping Rope 2 отличается тем, что позволяет перей..."
4,Крипта за глаза: Сэм Альтман запустил проект идентификации людей по радужке,"Сооснователь Open AI Сэм Альтман и его стартап Tools for Humanity объявили о запуске в США новой криптовалюты World (WLD). Получить её можно будет в обмен на сканирование радужной оболочки глаза, на основе которой сформируют уникальный hash-код. Этот код станет доказательством того, что владелец профиля World ID это человек, а не ИИ. Эта инициатива, первоначально называвшаяся Worldcoin, работа..."
